In [65]:
# Import our libraries
import pandas as pd
import numpy as np
# Import sklearn libraries
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve, auc, make_scorer, confusion_matrix, f1_score, fbeta_score
# Import the Naive Bayes, logistic regression, Bagging, RandomForest, AdaBoost, GradientBoost, Decision Trees and SVM Classifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import style
#plt.style.use('bmh')
#plt.style.use('ggplot')
plt.style.use('seaborn-notebook')
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer




train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")


data = [train_df, test_df]
for dataset in data:
    mean = train_df["Age"].mean()
    std = test_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_df["Age"].astype(int)
    
    
embarked_mode = train_df['Embarked'].mode()
data = [train_df, test_df]
for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(embarked_mode)

data = [train_df, test_df]
for dataset in data:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'travelled_alone'] = 'No'
    dataset.loc[dataset['relatives'] == 0, 'travelled_alone'] = 'Yes'
Y_train = train_df['Survived']

In [66]:
train_numerical_features = list(train_df.select_dtypes(include=['int64', 'float64', 'int32']).columns)
ss_scaler = StandardScaler()
train_df_ss = pd.DataFrame(data = train_df)
train_df_ss[train_numerical_features] = ss_scaler.fit_transform(train_df_ss[train_numerical_features])

test_numerical_features = list(test_df.select_dtypes(include=['int64', 'float64', 'int32']).columns)
ss_scaler = StandardScaler()
test_df_ss = pd.DataFrame(data = test_df)
test_df_ss[test_numerical_features] = ss_scaler.fit_transform(test_df_ss[test_numerical_features])




In [67]:
encode_col_list = list(train_df.select_dtypes(include=['object']).columns)
for i in encode_col_list:
    train_df_ss = pd.concat([train_df_ss,pd.get_dummies(train_df_ss[i], prefix=i)],axis=1)
    train_df_ss.drop(i, axis = 1, inplace=True)
    
    
    
    
encode_col_list = list(test_df.select_dtypes(include=['object']).columns)
for i in encode_col_list:
    test_df_ss = pd.concat([test_df_ss,pd.get_dummies(test_df_ss[i], prefix=i)],axis=1)
    test_df_ss.drop(i, axis = 1, inplace=True)
    
    
train_df_ss.drop(columns= 'Survived', axis = 1 , inplace = True )
X_train = train_df_ss[train_df_ss.columns[0:7].to_list()]
X_train = X_train.astype(float)


from sklearn.impute import SimpleImputer

imr = SimpleImputer(missing_values= np.NaN, strategy='mean')
imr = imr.fit(test_df_ss[['Fare']])
test_df_ss['Fare'] = imr.transform(test_df_ss[['Fare']]).ravel()


X_test = test_df_ss[test_df_ss.columns[0:7].to_list()]

X_test = X_test.astype(float)

In [68]:
# Instantiate our model
X_train = train_df_ss

logreg = LogisticRegression()
# Fit our model to the training data
#logreg.fit(X_train, Y_train)
# Predict on the test data
#logreg_predictions = logreg.predict(X_test)
#logreg_data = pd.read_csv('test.csv')
#logreg_data.insert((logreg_data.shape[1]),'Survived',logreg_predictions)
#logreg_data.to_csv('LogisticRegression_SS_OH_FE2.csv')

In [69]:
Y_pred = Y_pred.astype('int')
submit = pd.read_csv('gender_submission.csv')
submit['Survived'] = pd.Series(Y_pred)
submit.to_csv('KaggleTitanicSubmission.csv', index=False)